# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [75]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

In [76]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

nyc_data.db.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/nyc_data.db.zip'

con = duckdb.connect('nyc_data.db')

In [77]:
con = duckdb.connect('nyc_data.db')

In [78]:
con.install_extension('spatial')
con.load_extension('spatial')

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [79]:
con.sql("""
SELECT s.name, n.name, s.routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(
n.geom,
s.geom
)
WHERE n.name = 'Little Italy';
""")

┌───────────┬──────────────┬─────────┐
│   NAME    │     NAME     │ ROUTES  │
│  varchar  │   varchar    │ varchar │
├───────────┼──────────────┼─────────┤
│ Spring St │ Little Italy │ 6       │
└───────────┴──────────────┴─────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [80]:
con.sql("""
SELECT DISTINCT n.name, s.routes
FROM nyc_neighborhoods AS n
JOIN nyc_subway_stations AS s
ON ST_Contains(
n.geom,
s.geom
)
WHERE strpos(s.routes,'6') > 0;
""")

┌────────────────────┬─────────┐
│        NAME        │ ROUTES  │
│      varchar       │ varchar │
├────────────────────┼─────────┤
│ Financial District │ 4,5,6   │
│ Little Italy       │ 6       │
│ Upper East Side    │ 6       │
│ Upper East Side    │ 4,5,6   │
│ East Harlem        │ 6       │
│ East Harlem        │ 4,5,6   │
│ Soundview          │ 6       │
│ Parkchester        │ 6       │
│ Chinatown          │ 6       │
│ Greenwich Village  │ 6       │
│ Greenwich Village  │ 4,5,6   │
│ Gramercy           │ 6       │
│ Murray Hill        │ 4,5,6   │
│ Midtown            │ 6       │
│ Midtown            │ 4,5,6   │
│ Yorkville          │ 6       │
│ Mott Haven         │ 6       │
│ Hunts Point        │ 6       │
│ South Bronx        │ 6       │
├────────────────────┴─────────┤
│ 19 rows            2 columns │
└──────────────────────────────┘

In [81]:
#ANSWER
con.sql("""
SELECT DISTINCT n.name, n.boroname,
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE strpos(s.routes,'6') > 0;
""")

┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Financial District │ Manhattan │
│ Little Italy       │ Manhattan │
│ Upper East Side    │ Manhattan │
│ East Harlem        │ Manhattan │
│ Mott Haven         │ The Bronx │
│ Hunts Point        │ The Bronx │
│ South Bronx        │ The Bronx │
│ Chinatown          │ Manhattan │
│ Greenwich Village  │ Manhattan │
│ Gramercy           │ Manhattan │
│ Murray Hill        │ Manhattan │
│ Midtown            │ Manhattan │
│ Yorkville          │ Manhattan │
│ Soundview          │ The Bronx │
│ Parkchester        │ The Bronx │
├────────────────────┴───────────┤
│ 15 rows              2 columns │
└────────────────────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [82]:
con.sql("""

SELECT Sum(blk.popn_total)
FROM nyc_neighborhoods nh
JOIN nyc_census_blocks blk
ON ST_Intersects(nh.geom, blk.geom)
WHERE nh.name = 'Battery Park'

""")

┌─────────────────────┐
│ sum(blk.popn_total) │
│       int128        │
├─────────────────────┤
│               17153 │
└─────────────────────┘

4. **What neighborhood has the highest population density (persons/km2)?**


In [83]:
con.sql("PRAGMA table_info('nyc_census_blocks')").show()

┌───────┬────────────┬──────────┬─────────┬────────────┬─────────┐
│  cid  │    name    │   type   │ notnull │ dflt_value │   pk    │
│ int32 │  varchar   │ varchar  │ boolean │  varchar   │ boolean │
├───────┼────────────┼──────────┼─────────┼────────────┼─────────┤
│     0 │ BLKID      │ VARCHAR  │ false   │ NULL       │ false   │
│     1 │ POPN_TOTAL │ INTEGER  │ false   │ NULL       │ false   │
│     2 │ POPN_WHITE │ INTEGER  │ false   │ NULL       │ false   │
│     3 │ POPN_BLACK │ INTEGER  │ false   │ NULL       │ false   │
│     4 │ POPN_NATIV │ INTEGER  │ false   │ NULL       │ false   │
│     5 │ POPN_ASIAN │ INTEGER  │ false   │ NULL       │ false   │
│     6 │ POPN_OTHER │ INTEGER  │ false   │ NULL       │ false   │
│     7 │ BORONAME   │ VARCHAR  │ false   │ NULL       │ false   │
│     8 │ geom       │ GEOMETRY │ false   │ NULL       │ false   │
└───────┴────────────┴──────────┴─────────┴────────────┴─────────┘



In [84]:
con.sql("""
SELECT nyc_census_blocks.boroname, SUM(nyc_census_blocks.popn_total)/SUM(ST_Area(geom)) AS popn_density
FROM nyc_census_blocks
GROUP BY nyc_census_blocks.boroname
ORDER BY popn_density DESC
LIMIT 5;
""")

┌───────────────┬──────────────────────┐
│   BORONAME    │     popn_density     │
│    varchar    │        double        │
├───────────────┼──────────────────────┤
│ Manhattan     │   0.0268374186088613 │
│ Brooklyn      │ 0.013767572422878677 │
│ The Bronx     │  0.01256949174268445 │
│ Queens        │ 0.007876640287277638 │
│ Staten Island │ 0.003109148302933328 │
└───────────────┴──────────────────────┘

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [85]:
%pip install duckdb duckdb-engine jupysql

In [86]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [87]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [88]:
con.sql("""

SELECT * FROM read_csv_auto('https://storage.googleapis.com/qm2/casa0025_ships.csv')
LIMIT 5;

""")

┌──────────┬───────────────┬───────────────┬────────┬──────────┬─────────┬────────┬─────────────────────┬──────────────────┬──────────────────┬───────────────────────────────────────────┐
│ vesselid │  vessel_name  │   vsl_descr   │  dwt   │ v_length │ draught │  sog   │        date         │       lat        │       lon        │                   geom                    │
│  int64   │    varchar    │    varchar    │ double │  double  │ double  │ double │      timestamp      │      double      │      double      │                  varchar                  │
├──────────┼───────────────┼───────────────┼────────┼──────────┼─────────┼────────┼─────────────────────┼──────────────────┼──────────────────┼───────────────────────────────────────────┤
│   350053 │ 30 Let Pobedy │ general cargo │ 5150.0 │     NULL │     3.5 │    5.2 │ 2022-07-25 02:53:29 │ 45.1517766666667 │ 36.5133266666667 │ POINT (36.5133266666667 45.1517766666667) │
│   350053 │ 30 Let Pobedy │ general cargo │ 5150.0 │     NU

In [96]:
con.sql("""

CREATE OR REPLACE TABLE ais AS
SELECT
    vesselid,
    sog,
    date,
    lat,
    lon,
    ST_Point(lon, lat) AS geom
FROM read_csv_auto('https://storage.googleapis.com/qm2/casa0025_ships.csv')

""")

In [97]:
con.sql("CREATE INDEX ais_geom_idx ON ais USING RTREE(geom)")

In [91]:
con.sql("""

CREATE OR REPLACE TABLE vinfo AS
SELECT
    vesselid,
    vessel_name,
    vsl_descr,
    dwt,
    v_length,
    draught,
FROM read_csv_auto('https://storage.googleapis.com/qm2/casa0025_ships.csv')

""")

In [92]:
con.sql("CREATE INDEX vinfo_vesselid_idx ON vinfo(vesselid)")

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [99]:
con.sql("""
SELECT
a1.vesselid AS ship1,
a2.vesselid AS ship2,
a1.date AS start,
a2.date AS end
FROM ais AS a1
JOIN ais AS a2
ON ST_DWithin(a1.geom, a2.geom, 500)
WHERE
    ABS(EXTRACT(EPOCH FROM (a2.date - a1.date))) > 7200
    AND a1.vesselid <> a2.vesselid
""")

┌────────┬────────┬─────────────────────┬─────────────────────┐
│ ship1  │ ship2  │        start        │         end         │
│ int64  │ int64  │      timestamp      │      timestamp      │
├────────┼────────┼─────────────────────┼─────────────────────┤
│ 323648 │ 350053 │ 2022-06-28 14:31:37 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 14:41:56 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 14:51:59 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 15:02:17 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 15:24:19 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 16:12:20 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 18:06:21 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 19:06:25 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 20:12:20 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 21:12:22 │ 2022-07-25 02:53:29 │
│    ·   │    ·   │          ·          │          ·          │
│    ·   │    ·   │          ·          